## Resume parser utlity 

It can multipage resumes in process word (.doc & .docx) and pdf (.pdf) files

### Web Instance:

1. Resumes can be uploaded to local url
home url: http://127.0.0.1:5001
2. On submit , it redirects to output page with parsed resume data in form of table
output url: http://127.0.0.1:5001/output
3. It also generates two extract file of parsed tabular data (RESUME_DUMP.CSV, RESUME_DUMP.TXT)

Note: It can not process old word (.doc) file

### Local Instance

1. Resumes are placed in the directory indicated by constant "RESUME_FOLDER"
2. To enable local instance, uncomment local instance main method in below code (also disable the web instance main method at the same time)
3. Run the jupyter notebook to run resume parser
4. It also generates two extract file of parsed tabular data (RESUME_DUMP.CSV, RESUME_DUMP.TXT)

Note: It can process word (.doc & .docx) and pdf (.pdf) files. To process (.doc) file, it first converts them into (.docx) file.

### Library Dependency
1. pdfminer - to read (.pdf) file
2. docx2txt -  to read (.docx) file
3. spacy/nltk - for pattern matching and entity recognition


In [1]:
import io
import os
import re
import json
import docx2txt
from difflib import SequenceMatcher
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfparser import PDFParser
from glob import glob
import win32com.client as win32
from win32com.client import constants

In [2]:
import nltk
import spacy
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('maxent_ne_chunker')
#nltk.download('words')
import spacy
#!python -m spacy download en_core_web_sm
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
import en_core_web_sm

In [3]:
# FILE CONFIGURATIONS

# FILES UPLOADED TO FOLDER
#UPLOAD_FOLDER = "D:\DATA_SCIENCE\RESUME_PARSER\RESUME"
UPLOAD_FOLDER = "D:/DATA_SCIENCE/RESUME_PARSER/RESUME"
# FILES PLACED FOR LOCAL PARSING (USES DOUBLE BACK SLASH)
RESUME_FOLDER = "D:/DATA_SCIENCE/RESUME_PARSER/RESUME"
#RESUME_FOLDER = "D:\\DATA_SCIENCE\\RESUME_PARSER\\RESUME\\"

# Education degree list
EDUCATION_DB = "academic_degree_db.csv"

# External csv file with list of skills
SKILLS_DB = "skills_db.csv"

# External csv file with list of indian surnames
INDIAN_SURNAMES_DB = "indian_surname_db.csv"


In [4]:
# PATTERN CONFIGURATIONS

# NAME
NAME_PATTERN      = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]

# PHONE NUMBER
#PHONE_REG = re.compile(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]')
PHONE_REG = re.compile(r'([+(]?\d+[)\-]?[ \t\r\f\v]*[(]?\d{2,}[()\-]?[ \t\r\f\v]*\d{2,}[()\-]?[ \t\r\f\v]*\d*[ \t\r\f\v]*\d*[ \t\r\f\v]*)')

NOT_ALPHA_NUMERIC = r'[^a-zA-Z\d]'

NUMBER            = r'\d+'

# For finding date ranges
MONTHS_SHORT      = r'(jan)|(feb)|(mar)|(apr)|(may)|(jun)|(jul)|(aug)|(sep)|(oct)|(nov)|(dec)'
MONTHS_LONG       = r'(january)|(february)|(march)|(april)|(may)|(june)|(july)|(august)|(september)|(october)|(november)|(december)'
MONTH             = r'(' + MONTHS_SHORT + r'|' + MONTHS_LONG + r')'
YEAR              = r'(((20|19)(\d{2})))'

STOPWORDS         = set(stopwords.words('english'))

RESUME_SECTIONS = {
                    'education':['education','qualifications','academic','scholastic', 'certification'],
                    'skills':['skills','competencies','technical','technologies','skillset','training','proficiencies'],
                    'experience':['experience', 'professional experience','employment','job history','employment history','work experience','work history','professional experience','professional background']
                        }

In [5]:
def convert_doc_to_docx(path):
    # Create list of paths to .doc files
    #paths = glob(file_path+"//*.doc", recursive=True)
    # Opening MS Word
    word = win32.gencache.EnsureDispatch('Word.Application')
    doc = word.Documents.Open(path)
    doc.Activate ()
    # Rename path with .docx
    new_file_abs = os.path.abspath(path)
    new_file_abs = re.sub(r'\.\w+$', '.docx', new_file_abs)
    # Save and Close
    word.ActiveDocument.SaveAs(
        new_file_abs, FileFormat=constants.wdFormatXMLDocument
    )
    doc.Close(False)
    return

In [6]:
def extract_text_from_pdf(pdf_path):
    '''
    Extract text data from a pdf file
    input: pdf file path
    output: extracted text
    '''
    #print("Reading pdf document: ", pdf_path)
    
    output_string = io.StringIO()
    with open(pdf_path, 'rb') as in_file:
        parser = PDFParser(in_file)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(doc):
            interpreter.process_page(page)
        device.close()

    yield(output_string.getvalue())
    
#     with open(pdf_path, 'rb') as fh:
#         for page in PDFPage.get_pages(fh, caching=True, check_extractable=True):
#             resource_manager = PDFResourceManager()
#             fake_file_handle = io.StringIO()
#             converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
#             page_interpreter = PDFPageInterpreter(resource_manager, converter)
#             page_interpreter.process_page(page)
 
#             text = fake_file_handle.getvalue()
#             yield text
 
#             # close open handles
#             converter.close()
#             fake_file_handle.close()

In [7]:
def extract_text_from_doc(docx_path):
    '''
    Extract text data from a pdf file
    input: docx file path
    output: extracted text
    '''
    #print("Reading word document: ", docx_path)

    temp = docx2txt.process(docx_path)
    text = [line.replace('\t', ' ') for line in temp.split('\n') if line]
    return ' '.join(text)


In [8]:
def extract_text(file_path, extension):
    '''
    call different function based on file type to extract text
    '''
    text = ''
    if extension == '.pdf':
        for page in extract_text_from_pdf(file_path):
            text += ' ' + page
    elif extension == '.docx':
        text = extract_text_from_doc(file_path)
    else:
        pass
    text=str(text)
    if "CV" in text:
        text = text.replace("CV","")
    if "RESUME" in text:
        text = text.replace("RESUME","")
    if "Resume" in text:
        text = text.replace("Resume","")      
    if "curriculum vitae" in text:
        text = text.replace("curriculum vitae","")
    if "Curriculum Vitae" in text:
        text = text.replace("Curriculum Vitae","")    
    return text

In [9]:
def extract_resume_sections(text):
    '''
    extracts various sections, find start and end index of sections in resume text
    '''
    #text_split = [i.strip() for i in text.split('\n')]
    # sections_in_resume = [i for i in text_split if i.lower() in sections]
    section_labels = RESUME_SECTIONS
    end = len(text)-1
    sections={}
    sections['education']=[0,end]
    sections['skills']=[0,end]
    sections['experience']=[0,end]
    
    for value in section_labels.get('education'):
        start = text.find(value.upper())
        if start ==-1:
            start = text.find(value.title())
        if start != -1:
            sections['education']=[start,end]
    for value in section_labels.get('skills'):
        start = text.find(value.upper())
        if start ==-1:
            start = text.find(value.title())        
        if start != -1:
            sections['skills']=[start,end]
    for value in section_labels.get('experience'):
        start = text.find(value.upper())
        if start ==-1:
            start = text.find(value.title())
        if start != -1:
            sections['experience']=[start,end] 

    # update education index
    if sections.get('education')[0]<sections.get('skills')[0]:
        sections['education'][1] = sections.get('skills')[0]
        if (sections.get('education')[1]>sections.get('experience')[0]) and (sections.get('education')[0]<sections.get('experience')[0]):
            sections['education'][1] = sections.get('experience')[0]
            
    # update skills index
    if sections.get('skills')[0]<sections.get('education')[0]:
        sections['skills'][1] = sections.get('education')[0]
        if (sections.get('skills')[1]>sections.get('experience')[0]) and (sections.get('skills')[0]<sections.get('experience')[0]):
            sections['skills'][1] = sections.get('experience')[0]    
        
    # update experience index
    if sections.get('experience')[0]<sections.get('education')[0]:
        sections['experience'][1] = sections.get('education')[0]
        if (sections.get('experience')[1]>sections.get('skills')[0]) and (sections.get('experience')[0]<sections.get('skills')[0]):
            sections['experience'][1] = sections.get('skills')[0]   
            
    #print(sections)
    return sections

In [10]:
def extract_name(nlp_text, matcher, text, email):
    '''
    extract name from spacy nlp text and/or raw text 
    '''
    data = pd.read_csv(INDIAN_SURNAMES_DB)
    surname_bank = list(data.surname.values)
    start_name = ""  

    # extracting name from resume text which partially matched with email id
    text = re.sub("\S+@\S+(?:\.\S+)+", '', text)
    text = re.sub('[^A-Za-z\s]+', '', text).lower().split()
    wordlist = [word for word in text if word not in STOPWORDS]
    start_name = wordlist[0]+ " " + wordlist[1]
    
    if email:
        for i, word in enumerate(wordlist):
            try:
                first_name = word
                last_name = wordlist[i+1]
                name = first_name + " " + last_name
                if word:
                    if (SequenceMatcher(None, first_name, email).ratio()>0.3):
                        return (name)
            except:
                pass
    
    # extracting name from resume text that matches on the surnames list 
    for i, word in enumerate(wordlist):
        try:
            first_name = word
            last_name = wordlist[i+1]
            name = first_name + " " + last_name
            if (last_name in surname_bank) or (first_name in surname_bank):
                return name
        except:
            pass
    
    # extracting name using pattern match
    pattern = [NAME_PATTERN]
    matcher.add('NAME', None, *pattern)
    matches = matcher(nlp_text)
    if email:
        for match_id, start, end in matches:
            name = nlp_text[start:end].text
            if name:
                if (SequenceMatcher(None, name, email).ratio()>0.3):
                    return name

    return start_name


In [11]:
def extract_mobile_number(text):
    '''
    extract mobile number from text
    '''
    phone = re.findall(re.compile(r'(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?'), text)
    #phone = re.findall(re.compile(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]'))
    if phone:
        number = ''.join(phone[0])
        if len(number) > 10:
            return '+' + number
        else:
            return number

In [12]:
def extract_email(text):
    '''
    Retrieve email id from text
    '''
    email = re.findall("([^@|\s]+@[^@]+\.[^@|\s]+)", text.lower())
    if email:
        try:
            return email[0].split()[0].strip(';')
        except IndexError:
            return None

In [13]:
def extract_education(nlp_text):
    '''
    Extract education from spacy nlp text, output is tuple of education degree and/or year if year if found
    '''
    edu = {}
    flag = False
    degree_bank = pd.read_csv(EDUCATION_DB, index_col=0)
    # Extract education degree
    for index, text in enumerate(nlp_text):
        text = text.replace("!@#$%^&*()[]{};:,/<>?\|`~-=_+", "")
        
        for i in degree_bank.index:
            degree = str(degree_bank.iloc[i,0])
            if degree in text.upper():
                try:
                    edu[degree] = text + nlp_text[index + 1]
                    flag = True
                    break
                except:
                    edu[degree] = text
                    flag = True
                    break                
        if flag:
            break
            
#         for word in text.split():
#             #word = re.sub(r'[?|$|.|!|,]', r'', word)
#             if (word not in STOPWORDS):
#                 for degree in EDUCATION:
#                     if degree.find(word.upper()) != -1:
#                         try:
#                             edu[degree] = text + nlp_text[index + 1]
#                             flag = True
#                             break
#                         except:
#                             edu[degree] = text
#                             flag = True
#                             break

    # Extract year
    education = []
    for key in edu.keys():
        year = re.search(re.compile(YEAR), edu[key])
        if year:
            education.append((key, ''.join(year.group(0))))
        else:
            education.append(key)
    return education

In [14]:
def extract_skills(nlp_text, noun_chunks):
    '''
    extract skills from spacy nlp text, ouput is a list of skills
    '''
    tokens = [token.text for token in nlp_text if not token.is_stop]
    data = pd.read_csv(SKILLS_DB)
    skills_bank = list(data.columns.values)
    skillset = []
    # check for one-grams
    for token in tokens:
        if token.lower() in skills_bank:
            skillset.append(token)
    
    # check for bi-grams and tri-grams
    for token in noun_chunks:
        token = token.text.lower().strip()
        if token in skills_bank:
            skillset.append(token)
    return [i.capitalize() for i in set([i.lower() for i in skillset])]

In [15]:
class ResumeParser(object):
    def __init__(self, resume):
        #self.dict.
        #nlp = spacy.load("en_core_web_sm")
        nlp = en_core_web_sm.load()
        self.__matcher = Matcher(nlp.vocab)
        self.__details = {
            'name'              : None,
            'first_name'        : None,
            'last_name'         : None,
            'email'             : None,
            'mobile_number'     : None,
            'skills'            : None,
            'education'         : None
        }
        self.__resume      = resume
        self.__text_raw    = extract_text(self.__resume, os.path.splitext(self.__resume)[1])
        self.__text        = ' '.join(self.__text_raw.split()).replace("!#$%^&*()[]{}/<>?\|`~-=+", " ").replace('\u200b', ' ').strip()
        self.__sections    = extract_resume_sections(self.__text)
        self.__nlp         = nlp(re.sub('[^A-Za-z\s,;:]+', '', self.__text).lower())
        self.__nlp_edu     = nlp(self.__text[self.__sections["education"][0]:self.__sections["education"][1]])
        self.__noun_chunks = list(self.__nlp.noun_chunks)
        self.__get_basic_details()

    def get_extracted_data(self):
        return self.__details

    def __get_basic_details(self):
        email      = extract_email(self.__text)
        mobile     = extract_mobile_number(self.__text)
        #edu        = extract_education([sent.string.strip() for sent in self.__nlp_edu.sents])
        edu        = extract_education([sent.string.strip() for sent in self.__nlp.sents])
        skills     = extract_skills(self.__nlp, self.__noun_chunks)
        name       = extract_name(self.__nlp, matcher=self.__matcher, text=self.__text, email=email)
        if " " in name:
            first_name = name.split(" ")[:-1]
            last_name  = name.split(" ")[-1]
        else:
            first_name = name
            last_name = ""
        self.__details['name'] = " ".join(name) if type(name) is list else name
        self.__details['first_name'] = " ".join(first_name) if type(first_name) is list else first_name
        self.__details['last_name'] = last_name
        self.__details['email'] = email
        self.__details['mobile_number'] = mobile
        self.__details['education'] = ' '.join([str(elem) for elem in edu])
        self.__details['skills'] = ", ".join(skills) if type(skills) is list else skills
        #print(self.__sections)
        return

In [16]:
def convert_doc_to_docx(file_path):
    # doc_files = glob(file_path+'\\*.doc', recursive=True)

    # Opening MS Word
    word = win32.gencache.EnsureDispatch('Word.Application')
    doc = word.Documents.Open(file_path)
    doc.Activate ()

    # Rename path with .docx
    new_file_abs = os.path.abspath(file_path)
    new_file_abs = re.sub(r'\.\w+$', '.docx', new_file_abs)

    # Save and Close
    word.ActiveDocument.SaveAs(
        new_file_abs, FileFormat=constants.wdFormatXMLDocument
    )
    doc.Close(False)
    return

In [17]:
def bulk_convert_doc_to_docx(file_path):
    try:
        doc_files = glob(file_path+'\\*.doc', recursive=True)
        for doc_path in doc_files:
            print("File to convert: ", str(doc_path))
            convert_doc_to_docx(doc_path)
    except Exception as e: 
        print(e)
        pass
    return

In [18]:
def get_resume_details(file_path):
    resumes = []
    #print("current Path", str(file_path))
    bulk_convert_doc_to_docx(file_path)
    # Glob module matches certain patterns
    docx_files = glob(file_path+"\\*.docx")
    pdf_files = glob(file_path+"\\*.pdf")
    rtf_files = glob(file_path+"\\*.rtf")
    text_files = glob(file_path+"\\*.txt")

    files = set(docx_files + pdf_files + rtf_files + text_files)
    resumes = list(files)
    #print ("%d files identified" %len(files))
    resume_records = [ResumeParser(x).get_extracted_data() for x in resumes]
    #print(resumes)

    #df = pd.DataFrame(columns=["name", "first_name", "last_name", "email", "mobile_number", "education", "skills"])
    df = pd.DataFrame(columns=["first_name", "last_name", "email", "mobile_number", "education", "skills"])
    for i, resume in enumerate(resume_records):
        #print(pd.DataFrame(resume))
        #df.append(pd.DataFrame(resume), ignore_index = True)
        #df.loc[i,'name'] = resume.get('name', "")
        df.loc[i,'first_name'] = resume.get('first_name', "")
        df.loc[i,'last_name'] = resume.get('last_name', "")
        df.loc[i,'email'] = resume.get('email',"")
        df.loc[i,'mobile_number'] = resume.get('mobile_number',"")
        df.loc[i,'education'] = resume.get('education',"")
        df.loc[i,'skills'] = resume.get('skills',"")
        
    subdir = os.path.join(file_path, "OUTPUT")
    #print("Output directory:", subdir)
    if not os.path.exists(subdir):
        os.makedirs(subdir)
    
    df.to_csv(os.path.join(subdir, "RESUME_DUMP.CSV"), index=False)
    file1 = open(os.path.join(subdir, "RESUME_DUMP.TXT"),"w")
    #file1.write("\n")
    for line in resume_records:
        file1.writelines(str(json.dumps(line))+"\n")
        #file1.writelines(str(line.items())+"\n")
    file1.close()
    return df

In [19]:
import os
#import magic
import urllib.request
#from app import app
from flask import Flask, flash, request, redirect, render_template
from werkzeug.utils import secure_filename

ALLOWED_EXTENSIONS = set(['doc', 'docx', 'pdf'])

app = Flask(__name__)
app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
dir = os.path.join(UPLOAD_FOLDER)
#print("Output directory:", subdir)
if not os.path.exists(dir):
    os.makedirs(dir)
#app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/')
def upload_form():
    return render_template('upload_resume.html')

@app.route('/', methods=['POST'])
def upload_file():
    if request.method == 'POST':
        # check if the post request has the file part
        if 'files[]' not in request.files:
            flash('No file part')
            return redirect(request.url)
        
        files = request.files.getlist('files[]')

        for file in files:
            if file.filename == '':
                flash('No file selected for uploading')
                return redirect(request.url)
            if file and allowed_file(file.filename):
                filename = secure_filename(file.filename)
                file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
                flash('File(s) successfully uploaded')
                #if resp.status_code = 200:
            else:
                flash('Allowed file types are word and pdf')
                return redirect(request.url)
        return redirect('/output')

@app.route('/output')
def output():
    df = get_resume_details(RESUME_FOLDER)
    #print(df.head())
    #return render_template('home_resume.html', output_data=df.to_dict(orient='records'))
    return render_template('output_resume.html', tables=[df.to_html(classes='data', header="true", index=False)])

### Web Instance

In [ ]:
# FOR WEB BASED RESUME PARSING
# Displays parsed resume data on webpage, also generates "RESUME_DUMP.csv" and "RESUME_DUMP.txt"

if __name__ == "__main__":
    bulk_convert_doc_to_docx(RESUME_FOLDER)
    app.run(port=5001)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5001/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Mar/2021 21:40:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2021 21:40:13] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [30/Mar/2021 21:40:15] "GET /output HTTP/1.1" 200 -


### Local Instance

In [ ]:
# FOR LOCAL RESUME PARSING
# Generates output "RESUME_REPORT.csv" and "RESUME_REPORT.txt"

# if __name__ == "__main__":
#     bulk_convert_doc_to_docx(RESUME_FOLDER)
#     get_resume_details(RESUME_FOLDER)